In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fbprophet import Prophet
from datetime import datetime, timedelta

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler

ERROR:fbprophet:Importing plotly failed. Interactive plots will not work.


In [2]:
data=pd.read_csv('data/USD_KRW.csv')
data.columns=['ds','y_original']
data=data.set_index('ds')

## 정리 세팅(1번과 동일)

#### 1. 날짜 정리

In [3]:
start_date_train = ['2018-02-01','2018-03-01','2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01','2018-09-01',
           '2018-10-01','2018-11-01','2018-12-01','2019-01-01']
end_date_train = ['2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30',
           '2019-10-31','2019-11-30','2019-12-31']
start_date_test = ['2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
           '2019-10-01','2019-11-01','2019-12-01','2020-01-01']
end_date_test = ['2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30',
           '2019-10-31','2019-11-30','2019-12-31','2020-01-31']

#### 2. 휴일정리

In [4]:
holiday_list = [#2018 휴일
       '2018-01-01','2018-01-06','2018-01-07','2018-01-13','2018-01-14','2018-01-20','2018-01-21','2018-01-27',
       '2018-01-28','2018-02-03','2018-02-04','2018-02-10','2018-02-11','2018-02-15','2018-02-16','2018-02-17',
       '2018-02-18','2018-02-24','2018-02-25','2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
       '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
       '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
       '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
       '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       #2019 휴일
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
       '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
       '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
       '2019-11-23','2019-11-24','2019-11-30','2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
       '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29',
       #2020 휴일
       '2020-01-01','2020-01-04','2020-01-05','2020-01-11','2020-01-12','2020-01-18','2020-01-19','2020-01-24',
       '2020-01-25','2020-01-26','2020-01-27']

In [5]:
trh1 = ['2018-02-03','2018-02-04','2018-02-10','2018-02-11','2018-02-15','2018-02-16','2018-02-17',
       '2018-02-18','2018-02-24','2018-02-25','2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
       '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
       '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
       '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
       '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30','2019-01-01','2019-01-05','2019-01-06',
       '2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26','2019-01-27']
trh2 = ['2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
       '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
       '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
       '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
       '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24']
trh3 = ['2018-04-01','2018-04-07','2018-04-08',
       '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
       '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
       '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31'] 
trh4 = ['2018-05-05','2018-05-06',
       '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
       '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28']
trh5 = ['2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
       '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26'] 
trh6 = ['2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
       '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30']
trh7 = ['2018-08-04','2018-08-05','2018-08-12','2018-08-15',
       '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28']
trh8 = ['2018-09-01','2018-09-02','2018-09-08','2018-09-09',
       '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
       '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31']
trh9 = ['2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
       '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29']
trh10 = ['2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
       '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
       '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
       '2019-10-26','2019-10-27']
trh11 = ['2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
       '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
       '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
       '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
       '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
       '2019-11-23','2019-11-24','2019-11-30']
trh12 = ['2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
       '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
       '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
       '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
       '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
       '2019-11-23','2019-11-24','2019-11-30','2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
       '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29']
th1 = ['2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
       '2019-02-16','2019-02-17','2019-02-23','2019-02-24']
th2 = ['2019-03-01','2019-03-02','2019-03-03','2019-03-09',
       '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31']
th3 = ['2019-04-06',
       '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28']
th4 = ['2019-05-04',
       '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26']
th5 = ['2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
       '2019-06-23','2019-06-29','2019-06-30']
th6 = ['2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
       '2019-07-21','2019-07-27','2019-07-28']
th7 = ['2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
       '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31']
th8 = ['2019-09-01','2019-09-07','2019-09-08',
       '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29']
th9 = ['2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
       '2019-10-26','2019-10-27']
th10 = ['2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
       '2019-11-23','2019-11-24','2019-11-30']
th11 = ['2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
       '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29']
th12 = ['2020-01-01','2020-01-04','2020-01-05','2020-01-11','2020-01-12','2020-01-18','2020-01-19','2020-01-24',
       '2020-01-25','2020-01-26','2020-01-27']

In [6]:
train_holiday=[trh1,trh2,trh3,trh4,trh5,trh6,trh7,trh8,trh9,trh10,trh11,trh12]
test_holiday=[th1,th2,th3,th4,th5,th6,th7,th8,th9,th10,th11,th12]

In [7]:
df_lastday= pd.DataFrame(data=[1117.2, 1117.8, 1137.8, 1158.2, 1190.0, 1156.8, 1182.0, 1215.2, 1201.3, 1168.4, 1179.3, 1157.8],
                        columns=['rate'], index=['2019-01-31', '2019-02-28', '2019-03-29', '2019-04-30', '2019-05-31', '2019-06-28',
                                                 '2019-07-31', '2019-08-30', '2019-09-30', '2019-10-31', '2019-11-29', '2019-12-31'])

## 1번의 같은 모델과 코드를 활용함 -> 단지 시각화가 아닌 데이터프레임으로 만들기 위해서

#### set01

In [8]:
pred_price=[]

df_train=data.loc[start_date_train[0] : end_date_train[0], : ]
df_test=data.loc[start_date_test[0]: end_date_test[0], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[0]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[0] : end_date_train[0], : ]
future_test=future.loc[start_date_test[0]: end_date_test[0], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[0], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[0], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[0]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[0]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[0]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[0]:]))]
set01=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


#### set02

In [9]:
pred_price=[]

df_train=data.loc[start_date_train[1] : end_date_train[1], : ]
df_test=data.loc[start_date_test[1]: end_date_test[1], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[1]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[1] : end_date_train[1], : ]
future_test=future.loc[start_date_test[1]: end_date_test[1], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[1], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[1], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[1]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[1]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[1]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[1]:]))]
set02=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set03

In [10]:
pred_price=[]

df_train=data.loc[start_date_train[2] : end_date_train[2], : ]
df_test=data.loc[start_date_test[2]: end_date_test[2], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[2]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[2] : end_date_train[2], : ]
future_test=future.loc[start_date_test[2]: end_date_test[2], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[2], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[2], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[2]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[2]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[2]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[2]:]))]
set03=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set04

In [11]:
pred_price=[]

df_train=data.loc[start_date_train[3] : end_date_train[3], : ]
df_test=data.loc[start_date_test[3]: end_date_test[3], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[3]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[3] : end_date_train[3], : ]
future_test=future.loc[start_date_test[3]: end_date_test[3], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[3], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[3], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[3]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[3]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[3]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[3]:]))]
set04=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set05

In [12]:
pred_price=[]

df_train=data.loc[start_date_train[4] : end_date_train[4], : ]
df_test=data.loc[start_date_test[4]: end_date_test[4], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[4]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[4] : end_date_train[4], : ]
future_test=future.loc[start_date_test[4]: end_date_test[4], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[4], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[4], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[4]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[4]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[4]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[4]:]))]
set05=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set06

In [13]:
pred_price=[]

df_train=data.loc[start_date_train[5] : end_date_train[5], : ]
df_test=data.loc[start_date_test[5]: end_date_test[5], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[5]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[5] : end_date_train[5], : ]
future_test=future.loc[start_date_test[5]: end_date_test[5], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[5], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[5], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[5]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[5]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[5]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[5]:]))]
set06=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set07

In [14]:
pred_price=[]

df_train=data.loc[start_date_train[6] : end_date_train[6], : ]
df_test=data.loc[start_date_test[6]: end_date_test[6], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[6]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[6] : end_date_train[6], : ]
future_test=future.loc[start_date_test[6]: end_date_test[6], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[6], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[6], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[6]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[6]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[6]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[6]:]))]
set07=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set08

In [15]:
pred_price=[]

df_train=data.loc[start_date_train[7] : end_date_train[7], : ]
df_test=data.loc[start_date_test[7]: end_date_test[7], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[7]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[7] : end_date_train[7], : ]
future_test=future.loc[start_date_test[7]: end_date_test[7], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[7], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[7], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[7]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[7]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[7]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[7]:]))]
set08=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

##### set09

In [16]:
pred_price=[]

df_train=data.loc[start_date_train[8] : end_date_train[8], : ]
df_test=data.loc[start_date_test[8]: end_date_test[8], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[8]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[8] : end_date_train[8], : ]
future_test=future.loc[start_date_test[8]: end_date_test[8], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[8], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[8], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[8]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[8]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[8]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[8]:]))]
set09=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set10

In [17]:
pred_price=[]

df_train=data.loc[start_date_train[9] : end_date_train[9], : ]
df_test=data.loc[start_date_test[9]: end_date_test[9], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[9]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[9] : end_date_train[9], : ]
future_test=future.loc[start_date_test[9]: end_date_test[9], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[9], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[9], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[8]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[9]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[9]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[9]:]))]
set10=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set11

In [18]:
pred_price=[]

df_train=data.loc[start_date_train[10] : end_date_train[10], : ]
df_test=data.loc[start_date_test[10]: end_date_test[10], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[10]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[10] : end_date_train[10], : ]
future_test=future.loc[start_date_test[10]: end_date_test[10], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[10], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[10], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[10]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[10]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[10]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[10]:]))]
set11=pd.DataFrame([final_pred[:5], final_actual[:5]], columns=['day1','day2','day3','day4','day5'], index=['pred','actual']).T

#### set12

In [19]:
pred_price=[]

df_train=data.loc[start_date_train[11] : end_date_train[11], : ]
df_test=data.loc[start_date_test[11]: end_date_test[11], :]
df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1) train scale조정

y_original=np.array(df_train['y_original'])
sc=MinMaxScaler((0.05,0.95))
y=y_original.reshape(-1,1)
y = sc.fit_transform(y)
y_scale_train =pd.DataFrame(y)
y_scale_train.columns=['y']

df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
# 2)test scale 조정

y_scale_mean=df_train['y_original'].mean()
y_scale_std=df_train['y_original'].std()

df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

df_train=df_train.reset_index()
df_test=df_test.reset_index()

# 1. 예측값의 상한과 하한을 제어
df_train['cap'] = 1.0
df_train['floor'] = 0.0

# holiday이지만 주말과 공휴일을 다 삭제 
holiday = pd.DataFrame({
    'holiday': 'holiday',
    'ds': pd.to_datetime(
        train_holiday[11]), 
        'lower_window': 0,
        'upper_window': 0,
        })

m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
           interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 

m.fit(df_train)

# 3. 미래 Dataframe 생성
future = m.make_future_dataframe(periods=20)  # 예측날 설정
future.head()

future=future.set_index('ds')
future_train=future.loc[start_date_train[11] : end_date_train[11], : ]
future_test=future.loc[start_date_test[11]: end_date_test[11], :]

future_train=future_train.reset_index()
future_test=future_test.reset_index()


start_date = datetime.strptime(start_date_test[11], '%Y-%m-%d') #시작 날짜
end_date = datetime.strptime(end_date_test[11], '%Y-%m-%d') #끝 날짜

test_total_date = [] 

while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
    test_total_date.append(start_date.strftime('%Y-%m-%d')) 
    start_date += timedelta(days=1)


test_holiday_date=test_holiday[11]    #마찬가지로 주말, 공휴일 제거 추후 추가

final_date = list(set(test_total_date) - set(test_holiday_date))
future_test=future_test[future_test['ds'].isin(final_date)]
future=pd.concat([future_train,future_test],ignore_index=True)

future['cap'] = 1.0
future['floor'] = 0.0

#예측하기

forecast = m.predict(future)  
forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
pred_price.append(forecast['yhat'])

forecast=forecast.set_index('ds')

final_pred=[]
for i in sc.inverse_transform(forecast[start_date_test[11]:]):
    final_pred.append(round(i[0],2))
    
final_actual=[]
for i in df_test['y_original']:
    final_actual.append(i)
    
final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[11]:]))]

final_actual=final_actual[:len(sc.inverse_transform(forecast[start_date_test[11]:]))]
set12=pd.DataFrame([final_pred[:5], final_actual[:5]], index=['pred','actual']).T

## 5일만 예측하기에 예측값과 실제값 5개를 데이터 프레임으로 만들어줌

In [20]:
set01

,pred,actual
day1,1128.37,1111.6
day2,1130.63,1116.8
day3,1131.84,1123.4
day4,1131.89,1124.5
day5,1131.10,1124.5


In [21]:
set02

,pred,actual
day1,1115.23,1119.9
day2,1113.94,1125.0
day3,1113.77,1125.8
day4,1112.16,1127.8
day5,1111.95,1128.3


In [22]:
set03

,pred,actual
day1,1133.21,1136.1
day2,1133.19,1132.7
day3,1134.03,1135.8
day4,1133.87,1134.9
day5,1134.95,1135.5


In [23]:
set04

,pred,actual
day1,1156.16,1163.8
day2,1157.09,1163.9
day3,1159.25,1169.4
day4,1163.79,1169.8
day5,1165.48,1169.9


In [24]:
set05

,pred,actual
day1,1185.79,1191.2
day2,1185.40,1183.0
day3,1185.71,1181.5
day4,1186.83,1179.8
day5,1189.24,1179.5


In [25]:
set06

,pred,actual
day1,1160.48,1156.4
day2,1159.42,1155.4
day3,1159.13,1164.8
day4,1158.57,1170.4
day5,1158.89,1169.2


In [26]:
set07

,pred,actual
day1,1186.32,1181.9
day2,1186.90,1188.5
day3,1187.62,1194.8
day4,1187.63,1211.8
day5,1187.70,1214.3


In [27]:
set08

,pred,actual
day1,1218.02,1209.6
day2,1219.02,1211.9
day3,1219.83,1215.2
day4,1219.65,1209.7
day5,1220.35,1199.6


In [28]:
set09

,pred,actual
day1,1205.55,1199.2
day2,1207.91,1198.2
day3,1211.75,1203.7
day4,1216.65,1197.0
day5,1218.47,1196.5


In [29]:
set10

,pred,actual
day1,1163.40,1161.7
day2,1161.93,1167.6
day3,1161.36,1160.4
day4,1161.43,1160.1
day5,1161.12,1157.5


In [30]:
set11

,pred,actual
day1,1182.80,1180.3
day2,1183.94,1180.4
day3,1185.31,1186.2
day4,1186.78,1193.7
day5,1188.14,1190.2


In [31]:
set12

,pred,actual
0,1158.50,1157.8
1,1158.60,1156.9
2,1159.02,1162.7
3,1159.23,1169.6
4,1160.11,1165.3
